# **Pytorch - 1**
Membuat simple CNN classifier dengan dataset yang telah disediakan Pytorch.

Pertama mengimport library torch dan torchvision.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

Kemudian me-load dataset yang digunakan. Kali ini saya menggunakan dataset CIFAR-10.

In [7]:
transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

batch_size = 4

traindata = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(traindata, batch_size=batch_size, shuffle=True, num_workers=4)

testdata = torchvision.datasets.CIFAR10(root="./data", train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testdata, batch_size=batch_size, shuffle=False, num_workers=4)

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
